# PS7
Jingwen Fan

## 1a).


In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from pylab import rcParams
import warnings
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore") 
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
auto = pd.read_csv('Auto.csv', na_values='?')
auto.dropna(inplace=True)
auto.isnull().sum()


mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [11]:
auto['horsepower'] = auto['horsepower'].convert_objects(convert_numeric = True)
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= auto['mpg'].median() else 0).astype('category')
X = auto[['cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
y= auto['mpg_high'].values

In [12]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)
LogGeneral_error_0 = np.zeros(k)
LogGeneral_error_1 = np.zeros(k)
LogGeneral_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    print('When k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum() 
    LogGeneral_error_0[k_ind] = error_0
    LogGeneral_error_1[k_ind] = error_1
    LogGeneral_MSE[k_ind] = error_all_class
    print('\n',classification_report(y_test, y_pred, digits=3))
    print('error rate (category 0) is', error_0,
          ', error rate (category 1) is', error_1)
    print('The MSE of the model is', error_all_class)
    print()
    k_ind += 1


print('\nk-fold resuts:')
print('The average error rate (category 0) is', LogGeneral_error_0.mean(), ', std is', LogGeneral_error_0.std())
print('The average error rate (category 1) is', LogGeneral_error_1.mean(), ', std is', LogGeneral_error_1.std())
print('The average MSE of the model is', LogGeneral_MSE.mean(), ', std is', LogGeneral_MSE.std())

When k index= 0

              precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

error rate (category 0) is 0.057692307692307696 , error rate (category 1) is 0.13043478260869565
The MSE of the model is 0.09183673469387756

When k index= 1

              precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

error rate (category 0) is 0.12244897959183673 , error rate (category 1) is 0.08163265306122448
The MSE of the model is 0.10204081632653061

When k index= 2

              precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0.867     0.867        98

error rate (catego

## 1b)


In [14]:
forest = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
forest.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [16]:
forest.score(X, y)
MSE_df = pd.DataFrame({'ypred': forest.oob_decision_function_.T[1], 'y': y})
MSE_df['ypred'] = MSE_df['ypred'].apply(lambda x: 1 if int(x>=0.5) else 0)
MSE = mean_squared_error(MSE_df['y'], MSE_df['ypred'])
MSE_0 = mean_squared_error(MSE_df['y'].where(MSE_df.ypred<0.5).dropna(), MSE_df['ypred'].where(MSE_df.ypred<0.5).dropna())
MSE_1 = mean_squared_error(MSE_df['y'].where(MSE_df.ypred>0.5).dropna(), MSE_df['ypred'].where(MSE_df.ypred>0.5).dropna())
print('MSE=', MSE)
print('MSE for low category (0) = ', MSE_0)
print('MSE for high category (1) = ', MSE_1)

MSE= 0.07142857142857142
MSE for low category (0) =  0.05789473684210526
MSE for high category (1) =  0.08415841584158416


In [17]:

k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(X)
SVMGeneral_error_0 = np.zeros(k)
SVMGeneral_error_1 = np.zeros(k)
SVMGeneral_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    print('When k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum()
    SVMGeneral_error_0[k_ind] = error_0
    SVMGeneral_error_1[k_ind] = error_1
    SVMGeneral_MSE[k_ind] = error_all_class

    print('\n',classification_report(y_test, y_pred, digits=3))
    if (y_pred == 0).sum() != 0:
        print('error rate (category 0) is', error_0)
    else:
        print('error rate (category 0): NA (all predicted values are 1)')
    
    if (y_pred == 1).sum() != 0:
        print('error rate (category 1) is', error_1)
    else:
        print('error rate (category 1): NA (all predicted values are 0)')
    
    print('The MSE of the model is', error_all_class)
    print()
    k_ind += 1


print('\nk-fold resuts:')
SVMGeneral_error_0 = SVMGeneral_error_0[~np.isnan(SVMGeneral_error_0)]
print('The average error rate (category 0) is', SVMGeneral_error_0.mean(), ', std is', SVMGeneral_error_0.std())
SVMGeneral_error_1 = SVMGeneral_error_1[~np.isnan(SVMGeneral_error_1)]
print('The average error rate (category 1) is', SVMGeneral_error_1.mean(), ', std is', SVMGeneral_error_1.std())
print('The average MSE of the model is', SVMGeneral_MSE.mean(), ', std is', SVMGeneral_MSE.std())

When k index= 0

              precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

error rate (category 0) is 0.0
error rate (category 1) is 0.5520833333333334
The MSE of the model is 0.5408163265306123

When k index= 1

              precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

error rate (category 0) is 0.5204081632653061
error rate (category 1): NA (all predicted values are 0)
The MSE of the model is 0.5204081632653061

When k index= 2

              precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

error rate (category 0) is 0.5312

## 1d

Random forest is the best model with the lowest error rate and MSE, while SVM has the worst.